In [12]:
import pandas as pd
from datetime import datetime
import numpy as np
import requests
import json 
import os 

In [26]:
in_file = "../in_data/dossiers_journee-nationale-de-la-resilience-appel-a-projets_2023-07-26_07-58.xlsx"

df_dossier = pd.read_excel(in_file, sheet_name="Dossiers")
df_action_1 = pd.read_excel(in_file, sheet_name="(3344502) Action")
df_action_2 = pd.read_excel(in_file, sheet_name="(3308253) Action")

# Données INSEE
insee_data_dir = "C:\\Users\\sylvain.brisson\\Documents\\Données QGIS\\ADECOG_3-1_SHP_WGS84G_FRA"
df_dpt = pd.read_csv(os.path.join(insee_data_dir, "DEPARTEMENT.csv"))

In [27]:
df_dossier.set_index("ID", inplace=True)

# Création d'un ID pour les actions
for df in [df_action_1, df_action_2]:
    df["ID"] = df["Dossier ID"].astype(str) + "_" + df["Ligne"].astype(str)
    df.set_index("ID", inplace=True)

In [28]:
# Conversion des colonnes de dates en objet datetime 

for col in [
    'Dernière mise à jour le',
    'Déposé le',
    'Passé en instruction le',
    'Traité le'
] :
    df_dossier[col] = pd.to_datetime(df_dossier[col], format='%Y-%m-%d')

for col in [
    "Date prévisionnelle du début de l'action",
    "Date prévisionnelle de la fin de l'action",
] :
    for df in [df_action_1, df_action_2]:
        df[col] =  pd.to_datetime(df[col], format='%Y-%m-%d')
        

In [29]:
# Conversion des champs Oui/Non en True/False

for col in [
    '1. Une action se déroule dans une seule localisation ou est dématérialisée',
    '2 - Une ou plusieurs de vos actions se déroulent dans plusieurs localisations'
] :
    df_dossier[col] = df_dossier[col] == "Oui"

for col in [
    "L'action est-elle ouverte au grand public ?",
    'Votre action est-elle dématérialisée ?',
] :
    df_action_1[col] = df_action_1[col] == "Oui"

for col in [
    "L'action est-elle ouverte au grand public ?"
] :
    df_action_2[col] = df_action_2[col] == "Oui"


# Conversion des champs on/off en True/False

for col in [
    'En tant que porteur de projet proposé à la labellisation de l’Etat, je m’engage à respecter les valeurs et principes suivants, qui constituent la « Charte des valeurs de la Journée »',
    'Je m’engage, si mon projet est labellisé',
    "J'atteste sur l’honneur l’exactitude des renseignements fournis ;",
    "J'atteste avoir pris connaissance du règlement de l’appel à projets Journée nationale résilience 2023"
] :
    df_dossier[col] = df_dossier[col] == "on"

In [30]:
# Conversion des champs à choix multiples en plusieurs champs True / False

def traitement_champs_multiples(df, col, choix, fillna = False):
    
    # df[col] = df[col].fillna("")
    
    for c in choix : 
        df["Is "+c] = df.apply(
            lambda x : fillna if not(isinstance(x[col],str)) else (c in x[col].split(", ")),
            axis = 1
        )
        
    # df = df.drop(columns=[c])

    return df

In [31]:
for col,choix in [
    ("Type d'action", ["Conférence","Exposition","Réunion d'information","Visite en plein air","Atelier sensibilisation","Spectable","Autre","Formation","Conférence", "Atelier Jeux"]),
    ("Risques ciblés", ["Risques naturels","Risques technologiques"]),
    ("Risques naturels", ["Inondations","Feux de forêt","Tempête/cyclone","Séisme","Éruption volcanique","Mouvement de terrain","Risques littoraux","Avalanche","Radon"]),
    ("Risques technologiques", ["Accidents industriels","Accidents nucléaires","Rupture de barrage","Transport de matières dangereuses"]),
    ("Public cible", ["Tous public","Famille","Jeune public","Séniors"]),
    ("Public restreint", ["Elu","Scolaire","Salarié","Autres"]),
    ("Objectifs de l'action", ["Objectif n°1. Développer la culture sur les risques naturels et technologiques","Objectif n°2. Se préparer à la survenance d’une catastrophe","Objectif n°3. Développer la résilience collective aux catastrophes"])
] : 
    for df in [df_action_1, df_action_2]:
        df = traitement_champs_multiples(df, col, choix)

In [32]:
# Regarde si une action tombe en Octobre

dt1 = datetime(2023, 10, 1)
dt2 = datetime(2023, 10, 31)
dt_jnr = datetime(2023, 10, 13)

init_date = "Date prévisionnelle du début de l'action"
end_date = "Date prévisionnelle de la fin de l'action"

for df in [df_action_1, df_action_2]:
    
    df["Action en octobre"] = \
            ((df[init_date] >= dt1) & (df[init_date] <= dt2)) | \
            ((df[end_date] >= dt1) & (df[end_date] <= dt2)) | \
            ((df[end_date] >= dt2) & (df[init_date] <= dt1))
            
    df["Action journalière"] = (df[init_date] == df[init_date])
    df["Action JNR"] = (df["Action journalière"] & (df[init_date] == dt_jnr))
        

In [22]:
# Export des données par département

df_dossier["INSEE_DEP"] = df_dossier.apply(
    lambda row : row["Sélectionnez la zone géographique du projet"].split("-")[0],
    axis = 1
)

df_action_1 = pd.merge(df_action_1, df_dossier.reset_index()[["ID","INSEE_DEP"]], how="left", left_on="Dossier ID", right_on="ID")

col_risques = [f"Is {c}" for c in ["Inondations","Feux de forêt","Tempête/cyclone","Séisme"]]

In [24]:
df_nb_actions_risque = df_action_1.groupby("INSEE_DEP")[col_risques].sum()

nb_actions_risque_dict = {}

risques = [c[3:] for c in df_nb_actions_risque.columns]

couleurs_risques = {
    'Inondations' : "#74BFDF", 
    'Feux de forêt' : "#FC7C7C", 
    'Tempête/cyclone' : "#B4CAD3", 
    'Séisme' : "#D3C7B4"
}

couleurs = [couleurs_risques[r] for r in risques]

for code in df_dpt["INSEE_DEP"]:
    nb_actions_risque_dict[code] = None

for code in df_nb_actions_risque.index:
    data = df_nb_actions_risque.loc[code]
    nb_actions_risque_dict[code] = {
        "Type de risque" : risques,
        "Nombre d'actions" : data.values.flatten().tolist(),
        "couleur_plot" : couleurs
    }

with open('../out_data/nb_actions_dpt.json', 'w') as fp:
    json.dump(nb_actions_risque_dict, fp, indent=4)


In [33]:
# Récolte des coordonnées et département pour actions

def adress2latlon(adress, commune, dpt):
    
    """Interroge l'API de recherche d'adresse de OpenStreetMap
    - Si adresse non rentrée par le dépositaire du dossier OU non trouvée par OpenStreetMap : on cherche les coordonnées de la commune
    - Sinon on donne les coordonnées de l'adresse précise
    """
    
    if not(isinstance(adress, str)):
        return [np.nan,np.nan]
    
    url = f"https://nominatim.openstreetmap.org/?adressdetails=1&q={adress.replace(' ','+')}&format=json&limit=1"
    
    try :
        
        data = requests.get(url).json()[0]
                
        # commune = data["display_name"].split(", ")[2]
        # dpt = data["display_name"].split(", ")[4]

        return [data["lat"],data["lon"]]
    
    except :
        print(f"Warning : L'adresse suivante n'a pas été trouvée : {adress}, on donne les coordonnées de la commune")
        
        url = f"https://nominatim.openstreetmap.org/?adressdetails=1&q={commune.replace(' ','+')},+{dpt}&format=json&limit=1"

        try : 
            data = requests.get(url).json()[0]
            return [data["lat"],data["lon"]]
        
        except :
            print(f"Warning : Commune {commune} du département {dpt} non trouvée")
            return [np.nan,np.nan]
    
adress_col = "Veuillez indiquer l'adresse complète de l'action"
        
df_action_1[['Lat', 'Lon']] = pd.DataFrame([adress2latlon(adress,com,dpt) for adress,com,dpt in zip(df_action_1[adress_col],df_action_1["Commune"],df_action_1["Département"])],index=df_action_1.index)     

In [34]:
df_action_1 = pd.merge(df_action_1, df_dossier, how="left", left_on="Dossier ID", right_index=True)

In [35]:
# Export des données d'actions pour la carte grand public

df_action_1_export = df_action_1.loc[df_action_1["L'action est-elle ouverte au grand public ?"]][[
    "Nom de l'action",
    'Raison sociale',
    "Date prévisionnelle du début de l'action",
    "Date prévisionnelle de la fin de l'action",
    "Action journalière",
    "Action JNR",
    'Public cible',
    "Type d'action",
    "Description de l'action", 
    'Is Risques naturels',
    'Is Risques technologiques',
    'Risques naturels',
    'Risques technologiques',
    "Veuillez indiquer l'adresse complète de l'action",
    "Commune",
    "Département",
    "Lat",
    "Lon"
]].rename(columns={
    "Nom de l'action" : "Nom",
    "Raison sociale" : "Organisateur",
    "Action sur la JNR uniquement" : "Action_JNR",
    "Date prévisionnelle du début de l'action" : "Date_debut",
    "Date prévisionnelle de la fin de l'action" : "Date_fin",
    "Public cible" : "Public_cible",
    "Type d'action" : "Type_action",
    "Description de l'action" : "Descriptif",
    'Is Risques naturels' : "Is_risque_nat",
    'Is Risques technologiques' : "Is_risque_techno",
    'Risques naturels' : "Risques_nat",
    'Risques technologiques' : "Risques_techno",
    "Veuillez indiquer l'adresse complète de l'action" : "Adresse",
    "Action journalière" : "Action_journalière",
    "Action JNR" : "Action_JNR",
})

KeyError: "['Action sur la JNR uniquement'] not in index"

In [57]:
df_action_1_export[~df_action_1_export["Lat"].isna()].to_csv("../out_data/actions_grand_public_carte.csv")